In [ ]:
import pandas as pd
import numpy as np

from dataprep.eda import plot
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_1 = pd.read_csv("application_train.csv")
df_2 = pd.read_csv("application_test.csv")

### Quick EDA

In [ ]:
#eda semua data masih ada nilai NaN
plot(df_1)

In [ ]:
#menghapus nilai NaN
df_1.dropna(inplace=True)
df_2.dropna(inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
plot(df_1,"OCCUPATION_TYPE")

In [ ]:
plt.figure(figsize=[15,18])

features=['CODE_GENDER','NAME_EDUCATION_TYPE','OCCUPATION_TYPE','CNT_CHILDREN','NAME_FAMILY_STATUS','CNT_FAM_MEMBERS']
n=1

for f in features:
    plt.subplot(3,2,n)
    order=df_1[f].value_counts().index
    sns.countplot(y=df_1[f],order=order);
    sns.despine()
    plt.title("Countplot of {}".format(f))
    n=n+1
plt.tight_layout()

In [ ]:
plot(df_1,"TARGET")

In [ ]:
plot(df_1,"AMT_CREDIT")

memangkas data yang tidak diperlukan merujuk ke data AMT_CREDIT [Credit amount of the loan] atau tren jumlah pinjaman yang di pinjam oleh peminjam karna distribusi menujukan bahwa kebanyakan peminjam meminjam tidak lebih dari 900k

In [ ]:
train= df_1.drop(['DAYS_BIRTH','DAYS_REGISTRATION','DAYS_ID_PUBLISH','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG','ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','FLOORSMIN_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE','YEARS_BUILD_MODE','ELEVATORS_MODE','ENTRANCES_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI','FONDKAPREMONT_MODE','HOUSETYPE_MODE','TOTALAREA_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','LANDAREA_AVG','COMMONAREA_MODE','HOUR_APPR_PROCESS_START'],axis=1)

### Data Prepocessing

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X=train.drop(columns=['SK_ID_CURR','OCCUPATION_TYPE','TARGET'],axis=1)
y=train['TARGET']

In [ ]:
#numerical columns
num=X.select_dtypes(exclude='object').columns

#categorical columns
cat=X.select_dtypes(include='object').columns

In [ ]:
impute=SimpleImputer(strategy='most_frequent')

Xc=impute.fit_transform(X[cat])

In [ ]:
encoding=OneHotEncoder()

Xc=encoding.fit_transform(Xc)

In [ ]:
transform=PowerTransformer()

Xn=transform.fit_transform(X[num])

### Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

In [ ]:
fs=SelectKBest(score_func=chi2,k='all')
fs.fit(Xc,y)
#chi square method is used to test for independence of categorical variables

plt.figure(figsize=(12,8))
plt.bar([i for i in range(len(fs.scores_))],fs.scores_);

### Model Evaluation Metric

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
def evaluate(X,y,model):
    cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return scores

### Test to determine k parameter for feature selection ###

In [ ]:
results=[]
Xc=Xc.toarray()

for i in range(15,30):
    fs=SelectKBest(score_func=chi2,k=i)
    fs.fit(Xc,y)
    
    Xfs=fs.transform(Xc)
    X=np.concatenate([Xn,Xfs],axis=1)
    
    results.append(evaluate(X,y,LogisticRegression()))
    
plt.figure(figsize=(12,8))
plt.boxplot(results,labels=range(15,30),showmeans=True);

In [ ]:
fs=SelectKBest(score_func=chi2,k=23)
fs.fit(Xc,y)

Xfs=fs.transform(Xc)
X=np.concatenate([Xn,Xfs],axis=1)

X.shape

### SPOT CHECKING ALGHORITHM

In [ ]:
def get_models():
    
    models=[]
    names=[]
    
    models.append(LogisticRegression())
    names.append('lr') 
                  
    models.append(RandomForestClassifier())
    names.append('rf')              
                  
    models.append(KNeighborsClassifier())
    names.append('knn')
    
    models.append(GaussianNB())
    names.append('nb')
    
    models.append(AdaBoostClassifier())
    names.append('adb')
    
    models.append(GradientBoostingClassifier())
    names.append('grb')

    return models,names

In [ ]:
results=[]
model,names=get_models()

for i in range(len(model)):
    results.append(evaluate(X,y,model[i]))
    
for i in range(len(results)):
    print(f'{names[i]}: {results[i].mean()}, {(results[i].std())}')

In [ ]:
plt.figure(figsize=(12,8))
plt.boxplot(results,labels=names,showmeans=True);

**Correcting Class Imbalance**

In [ ]:
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

In [ ]:
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_res,y_res=resample.fit_resample(X,y)

results=[]
model,names=get_models()

for i in range(len(model)):
    results.append(evaluate(X_res,y_res,model[i]))
    
plt.figure(figsize=(12,8))
plt.boxplot(results,labels=names,showmeans=True);

Hyperparameter Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters={'n_estimators':range(150,300,20),
            'max_features':[0.4,0.5,0.6,0.7,0.8]}

In [ ]:
model=RandomForestClassifier()
cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

grid=GridSearchCV(model,parameters,scoring='roc_auc',cv=cv,n_jobs=-1)
grid.fit(X_res,y_res)

In [ ]:
grid.best_score_

In [ ]:
params=grid.best_params_

final model pipline

In [ ]:
inter=Pipeline([('i',SimpleImputer(strategy='most_frequent')),
                ('e',OneHotEncoder()),
                ('fs',SelectKBest(score_func=chi2,k=23))])
              
transformers=[('t',PowerTransformer(),num),
              ('inter',inter,cat)]
    
steps=[('pre',ColumnTransformer(transformers=transformers)),
       ('sampling',SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))),
       ('model',RandomForestClassifier(**params))]

In [ ]:
X_train=train.drop(columns=['SK_ID_CURR','OCCUPATION_TYPE','TARGET'],axis=1)
y_train=train['TARGET']

X_test=df_2.drop(columns=['SK_ID_CURR','OCCUPATION_TYPE'],axis=1)

In [ ]:
model=Pipeline(steps=steps)
model.fit(X_train,y_train)

predictions=model.predict(X_test)

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR':test['SK_ID_CURR'],'TARGET':predictions})
submission.head(10)